In [1]:
%load_ext Cython

In [2]:
from scipy.stats import gumbel_r

shocks=[gumbel_r.rvs() for i in range(50000)]

counter_pos=0
counter_neg=0
total={}
for element in shocks:
    if element>0:
        counter_pos+=1
    else: counter_neg+=1
total['pos']=counter_pos
total['neg']=counter_neg

print total

{'neg': 18366, 'pos': 31634}


In [3]:
%%cython
cimport cython
from libc.stdlib cimport calloc, free, rand, RAND_MAX
from cython.parallel import prange, parallel

cpdef unsigned int rand_state(unsigned int min, unsigned int max) nogil:
    cdef double scaled
    #Generate the random number between 0 and 1
    scaled=rand()/<double>RAND_MAX
    return <unsigned int>(max*scaled + min)

# @cython.boundscheck(False)
# @cython.wraparound(False)
# cdef void* FILL_int(unsigned int *arr, size_t rows, size_t cols) nogil:
#     cdef Py_ssize_t i,j
#     with parallel(num_threads=4):
#         for i in prange(rows, schedule='dynamic'):
#             for j in range(cols):
#                 arr[j+i*cols]=0  
                
@cython.boundscheck(False)
@cython.wraparound(False)
cpdef unsigned int SUMS(size_t HH, size_t Gen) nogil:
    
    cdef:
        unsigned int *decisions
        Py_ssize_t i, j
        unsigned int total=0
    
    decisions=<unsigned int*> calloc(HH*Gen, sizeof(unsigned int))
    
    #with parallel(num_threads=4):
    
    for j in xrange(HH):#, schedule='dynamic'):
        decisions[j]=rand_state(1,4)

    for i in xrange(HH):#, schedule='dynamic'):
        for j in range(Gen):
            total+=decisions[j+i*Gen]
            
    free(decisions)
    return total  

In [4]:
print SUMS(1000,1)

2515


In [5]:
## Experimental Cython Functions to calculate the Next State based on the transition functions

In [6]:
%%cython

#Prototype Cython function to generate the next state based on the transition function probabilities 
#flexible enough to work with either a deterministic or non deterministic transition function

cimport cython
from libc.stdlib cimport calloc, free, rand, RAND_MAX

cdef size_t tot_states=4

cdef inline double rand_value() nogil:
    return rand()/<double>RAND_MAX

#This function rewrites the element in an array with the cumulative sum of the components
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
cdef void cum_sum(double *arr, size_t index=4-1) nogil:
    if index<=0: return
    cum_sum(&arr[0], index-1)
    arr[index]+=arr[index-1]

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
cdef unsigned int find_interval(double x, double *arr) nogil:
    cdef Py_ssize_t i
    
    for i in range(tot_states):
        if x<arr[i]:
            return i
            
    return 0

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
cpdef unsigned int next_state(double[:] tran):
    cdef:
        double x
        double *array
        unsigned int index
        Py_ssize_t i
        
    array=<double*> calloc(tot_states, sizeof(double))
    
#     with gil:
    if not array: raise MemoryError()
        
    try:
        x = rand_value()

        for i in range(tot_states):
            array[i]=tran[i]

        cum_sum(&array[0])
       
        index = find_interval(x, &array[0]) + 1
    
        return index
        
    finally:
        free(array)     
        

In [7]:
import numpy as np

tot_states=4    #number of states
tot_decisions=4 #number of decisions

tran_st=[[1,0,0,0],[0.3799,0,0.6201,0],[0,1,0,0],[0,0.3496,0,0.6504],
         [0,1,0,0],[0,0.5125,0,0.4875],[1,0,0,0],[0.3964,0,0.6036,0],
         [1,0,0,0],[0.2635,0,0.7365,0],[0,1,0,0],[0,0.1365,0,0.8635],
         [0,1,0,0],[0,0.2078,0,0.7922],[1,0,0,0],[0.1429,0,0.8571,0]]

tran_func=np.array(tran_st, dtype='d').reshape((tot_states,tot_decisions,tot_states))

state=2
decision=2

In [8]:
print tran_func

[[[ 1.      0.      0.      0.    ]
  [ 0.3799  0.      0.6201  0.    ]
  [ 0.      1.      0.      0.    ]
  [ 0.      0.3496  0.      0.6504]]

 [[ 0.      1.      0.      0.    ]
  [ 0.      0.5125  0.      0.4875]
  [ 1.      0.      0.      0.    ]
  [ 0.3964  0.      0.6036  0.    ]]

 [[ 1.      0.      0.      0.    ]
  [ 0.2635  0.      0.7365  0.    ]
  [ 0.      1.      0.      0.    ]
  [ 0.      0.1365  0.      0.8635]]

 [[ 0.      1.      0.      0.    ]
  [ 0.      0.2078  0.      0.7922]
  [ 1.      0.      0.      0.    ]
  [ 0.1429  0.      0.8571  0.    ]]]


In [9]:
print next_state(tran_func[state-1,decision-1,:])

4


In [12]:
#test that this Cython function is producing states according to the probability weights
from collections import Counter

lst=[]
loop=100000
for i in range(loop):
    lst.append(next_state(tran_func[1,3,:]))
    
c= Counter(lst)
for key in c:
    c[key] = c[key] / float(loop)

print c

Counter({3: 0.60572, 1: 0.39428})


In [11]:
%timeit next_state(tran_func[1,3,:])

The slowest run took 10.51 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 1.25 µs per loop


In [ ]:
%%cython

#A comparison of the low-level dot product from fortran with numpy (and how to use the 
#low-level BLAS functions)

#Global Cython Compiler Directives:

#!python
#cython: boundscheck=False
#cython: wraparound=False
#cython: nonecheck=False
#cython: cdivision=True

#Import modules
import scipy.linalg.blas

from cpython cimport (PY_VERSION_HEX, PyCObject_Check,
    PyCObject_AsVoidPtr, PyCapsule_CheckExact, PyCapsule_GetPointer)

#Define the fortran pointer used for the BLAS functions
cdef void* f2py_pointer(obj):
    if PY_VERSION_HEX < 0x03000000:
        if (PyCObject_Check(obj)):
            return PyCObject_AsVoidPtr(obj)
    elif PY_VERSION_HEX >= 0x02070000:
        if (PyCapsule_CheckExact(obj)):
            return PyCapsule_GetPointer(obj, NULL);
    raise ValueError("Not an object containing a void ptr")
    
#Define 
ctypedef int dgemm_t(
    char *transa, char *transb,
    int *m, int *n, int *k,
    double *alpha,
    double *a, int *lda,
    double *b, int *ldb,
    double *beta,
    double *c, int *ldc)

# Since Scipy >= 0.12.0
cdef dgemm_t *dgemm = <dgemm_t*>f2py_pointer(scipy.linalg.blas.dgemm._cpointer)

cpdef myfunc():
    cdef: 
        double* a 
        double* b 
        double* c
        int m, n, k, lda, ldb, ldc
        double alpha, beta

    a = <double*> malloc(m*k * sizeof(double))
    b = <double*> malloc(k*n * sizeof(double))
    c = <double*> calloc(m*n, sizeof(double))
    
    alpha = 1.0
    beta = 0.0
    lda = 2
    ldb = 2
    ldc = 2
    m = 2
    n = 2
    k = 2
    dgemm("N", "N", &m, &n, &k, &alpha, &a[0,0], &lda, &b[0,0], &ldb, &beta, &c[0,0], &ldc)
    print(np.asarray(c))
    print(np.dot(a, b))

In [1]:
%load_ext pymatbridge

/usr/local/lib/python2.7/site-packages/IPython/nbformat.py:13: ShimWarning: The `IPython.nbformat` package has been deprecated. You should import from nbformat instead.
  "You should import from nbformat instead.", ShimWarning)


Starting MATLAB on ZMQ socket ipc:///tmp/pymatbridge-5eb011b6-bc97-403b-b0d4-15ed96016fe1
Send 'exit' command to kill the server
.......MATLAB started and connected!
